In [ ]:
import openai
import os
import time
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve environment variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ASSISTANT_ID = os.getenv("ASSISTANT_ID")

# Verify that the environment variables are set
if not OPENAI_API_KEY or not ASSISTANT_ID:
    raise EnvironmentError("Environment variables OPENAI_API_KEY and ASSISTANT_ID must be set.")

# Set the OpenAI API key
openai.api_key = OPENAI_API_KEY


def create_thread():
    """Creates a new thread."""
    try:
        thread = openai.beta.threads.create()
        return thread.id
    except Exception as e:
        raise RuntimeError(f"Error creating thread: {e}")


def add_message_to_thread(thread_id, user_message):
    """Adds a user message to a thread."""
    try:
        openai.beta.threads.messages.create(
            thread_id=thread_id,
            role="user",
            content=user_message
        )
    except Exception as e:
        raise RuntimeError(f"Error adding message to thread: {e}")


def run_assistant(thread_id):
    """Runs the assistant and waits for the response."""
    try:
        run = openai.beta.threads.runs.create(
            thread_id=thread_id,
            assistant_id=ASSISTANT_ID
        )
        # Wait for the run to complete
        while run.status != 'completed':
            time.sleep(2)
            run = openai.beta.threads.runs.retrieve(
                thread_id=thread_id,
                run_id=run.id
            )
    except Exception as e:
        raise RuntimeError(f"Error running assistant: {e}")


def get_assistant_response(thread_id):
    """Retrieves the assistant's response from the thread."""
    try:
        messages = openai.beta.threads.messages.list(thread_id=thread_id)
        for msg in messages.data:
            if msg.role == "assistant":
                return msg.content[0].text.value
        return "No response from assistant."
    except Exception as e:
        raise RuntimeError(f"Error retrieving assistant response: {e}")


def main():
    """Main function to handle user interaction with the assistant."""
    try:
        thread_id = create_thread()
        user_message = input("Enter your question for the assistant: ")
        add_message_to_thread(thread_id, user_message)
        run_assistant(thread_id)
        response = get_assistant_response(thread_id)
        print(f"\nAssistant Response: {response}")
    except Exception as e:
        print(f"An error occurred: {e}")


if __name__ == "__main__":
    main()